In [51]:
import pandas as pd
import numpy as np
import plotly_express as px
import seaborn as sns
import matplotlib.pyplot as plt

In [52]:
df = pd.read_csv("DataEngineer.csv")
df.head()

,Job Title,Salary Estimate,Job Description,Rating,Company Name,Location,Headquarters,Size,Founded,Type of ownership,Industry,Sector,Revenue,Competitors,Easy Apply
0,Data Engineer,$80K-$150K (Glassdoor est.),Company Description\r\nSagence is a management...,4.5,Sagence\r\n4.5,"New York, NY","Chicago, IL",1 to 50 employees,2009,Company - Private,Consulting,Business Services,$10 to $25 million (USD),"WCI Consulting, PwC",-1
1,Senior Data Engineer (Healthcare Domain experi...,$80K-$150K (Glassdoor est.),"Key Responsibilities\r\n\r\n- Architect, build...",3.4,Enterprise Integration\r\n3.4,"New York, NY","Jacksonville, FL",51 to 200 employees,1998,Company - Private,IT Services,Information Technology,$25 to $50 million (USD),-1,-1
2,Data Engineers,$80K-$150K (Glassdoor est.),Overview\r\n\r\nJob description\r\n\r\nPositio...,5.0,Maestro Technologies\r\n5.0,"New York, NY","Trenton, NJ",51 to 200 employees,2003,Company - Private,IT Services,Information Technology,$5 to $10 million (USD),-1,-1
3,Client Trade Support Engineer,$80K-$150K (Glassdoor est.),About the Position\r\n\r\n\r\nThis position wi...,4.8,Jane Street\r\n4.8,"New York, NY","New York, NY",501 to 1000 employees,2000,Company - Private,Investment Banking & Asset Management,Finance,Unknown / Non-Applicable,-1,-1
4,Data Engineer,$80K-$150K (Glassdoor est.),Data Engineer\r\n\r\nJob Details\r\nLevel\r\nE...,3.7,GNY Insurance Companies\r\n3.7,"New York, NY","New York, NY",201 to 500 employees,1914,Company - Private,Insurance Carriers,Insurance,$100 to $500 million (USD),"Travelers, Chubb, Crum & Forster",True


In [53]:
df.shape

(2528, 15)

In [54]:
df.columns

Index(['Job Title', 'Salary Estimate', 'Job Description', 'Rating',
       'Company Name', 'Location', 'Headquarters', 'Size', 'Founded',
       'Type of ownership', 'Industry', 'Sector', 'Revenue', 'Competitors',
       'Easy Apply'],
      dtype='object')

In [55]:
x = df.mean()
print(x)

Rating        3.379549
Founded    1534.771756
dtype: float64


C:\Users\Lenovo\AppData\Local\Temp\ipykernel_15788\4224734047.py:1: FutureWarning:

Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.



In [56]:
df.replace('-1', np.nan, inplace=True)
df.replace(-1, np.nan, inplace=True)


In [57]:
df.isnull().mean()

Job Title            0.000000
Salary Estimate      0.000000
Job Description      0.000000
Rating               0.092563
Company Name         0.000000
Location             0.000000
Headquarters         0.049051
Size                 0.047073
Founded              0.225870
Type of ownership    0.047073
Industry             0.149525
Sector               0.148339
Revenue              0.047073
Competitors          0.691851
Easy Apply           0.942247
dtype: float64

In [58]:
df.drop(columns=['Job Description', 'Company Name', 'Competitors', 'Easy Apply', 'Competitors'])


,Job Title,Salary Estimate,Rating,Location,Headquarters,Size,Founded,Type of ownership,Industry,Sector,Revenue
0,Data Engineer,$80K-$150K (Glassdoor est.),4.5,"New York, NY","Chicago, IL",1 to 50 employees,2009.0,Company - Private,Consulting,Business Services,$10 to $25 million (USD)
1,Senior Data Engineer (Healthcare Domain experi...,$80K-$150K (Glassdoor est.),3.4,"New York, NY","Jacksonville, FL",51 to 200 employees,1998.0,Company - Private,IT Services,Information Technology,$25 to $50 million (USD)
2,Data Engineers,$80K-$150K (Glassdoor est.),5.0,"New York, NY","Trenton, NJ",51 to 200 employees,2003.0,Company - Private,IT Services,Information Technology,$5 to $10 million (USD)
3,Client Trade Support Engineer,$80K-$150K (Glassdoor est.),4.8,"New York, NY","New York, NY",501 to 1000 employees,2000.0,Company - Private,Investment Banking & Asset Management,Finance,Unknown / Non-Applicable
4,Data Engineer,$80K-$150K (Glassdoor est.),3.7,"New York, NY","New York, NY",201 to 500 employees,1914.0,Company - Private,Insurance Carriers,Insurance,$100 to $500 million (USD)
...,...,...,...,...,...,...,...,...,...,...,...
2523,Cloud Engineer,$76K-$128K (Glassdoor est.),4.5,"Westlake, TX","Herndon, VA",1 to 50 employees,NaN,Company - Private,Computer Hardware & Software,Information Technology,$1 to $5 million (USD)
2524,Electrical Engineer,$76K-$128K (Glassdoor est.),2.9,"Fort Worth, TX","Santa Clara, CA",51 to 200 employees,1993.0,Company - Private,Staffing & Outsourcing,Business Services,$5 to $10 million (USD)
2525,Sr. Big Data Engineer,$76K-$128K (Glassdoor est.),3.7,"Irving, TX","Cincinnati, OH",501 to 1000 employees,NaN,Company - Private,IT Services,Information Technology,$100 to $500 million (USD)
2526,Senior Big Data / ETL Engineer,$76K-$128K (Glassdoor est.),3.5,"Westlake, TX","Dettenheim, Germany",Unknown,NaN,Company - Public,NaN,NaN,Unknown / Non-Applicable


In [74]:
sestimate = df.groupby(['Salary Estimate']).size().reset_index(name = 'Count').sort_values('Count', ascending=False)
sestimate.replace(regex='\(Glassdoor est.\)', value='', inplace=True)
sestimate.replace(regex='\(Employer est.\)', value='', inplace=True)
sestimate.replace(regex='K', value='000', inplace=True)
sestimate.replace(regex='\$', value='', inplace=True)
sestimate['Salary Estimate mean'] = sestimate['Salary Estimate'].str.split('-', expand=True).astype(int).mean(axis=1)
px.box(sestimate, x='Salary Estimate mean', title='Boxplot of Salary Estimate\'s mean')

In [60]:
industry = df.groupby(['Industry']).size().reset_index(name = 'Count').sort_values('Count', ascending=False)

px.bar(industry, x='Industry', y='Count')


In [61]:
df['Type of ownership'].replace(regex='Company -', value='', inplace=True)
df


,Job Title,Salary Estimate,Job Description,Rating,Company Name,Location,Headquarters,Size,Founded,Type of ownership,Industry,Sector,Revenue,Competitors,Easy Apply
0,Data Engineer,$80K-$150K (Glassdoor est.),Company Description\r\nSagence is a management...,4.5,Sagence\r\n4.5,"New York, NY","Chicago, IL",1 to 50 employees,2009.0,Private,Consulting,Business Services,$10 to $25 million (USD),"WCI Consulting, PwC",NaN
1,Senior Data Engineer (Healthcare Domain experi...,$80K-$150K (Glassdoor est.),"Key Responsibilities\r\n\r\n- Architect, build...",3.4,Enterprise Integration\r\n3.4,"New York, NY","Jacksonville, FL",51 to 200 employees,1998.0,Private,IT Services,Information Technology,$25 to $50 million (USD),NaN,NaN
2,Data Engineers,$80K-$150K (Glassdoor est.),Overview\r\n\r\nJob description\r\n\r\nPositio...,5.0,Maestro Technologies\r\n5.0,"New York, NY","Trenton, NJ",51 to 200 employees,2003.0,Private,IT Services,Information Technology,$5 to $10 million (USD),NaN,NaN
3,Client Trade Support Engineer,$80K-$150K (Glassdoor est.),About the Position\r\n\r\n\r\nThis position wi...,4.8,Jane Street\r\n4.8,"New York, NY","New York, NY",501 to 1000 employees,2000.0,Private,Investment Banking & Asset Management,Finance,Unknown / Non-Applicable,NaN,NaN
4,Data Engineer,$80K-$150K (Glassdoor est.),Data Engineer\r\n\r\nJob Details\r\nLevel\r\nE...,3.7,GNY Insurance Companies\r\n3.7,"New York, NY","New York, NY",201 to 500 employees,1914.0,Private,Insurance Carriers,Insurance,$100 to $500 million (USD),"Travelers, Chubb, Crum & Forster",True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2523,Cloud Engineer,$76K-$128K (Glassdoor est.),The Expertise You Have Proven expertise in dev...,4.5,PMG Global\r\n4.5,"Westlake, TX","Herndon, VA",1 to 50 employees,NaN,Private,Computer Hardware & Software,Information Technology,$1 to $5 million (USD),NaN,NaN
2524,Electrical Engineer,$76K-$128K (Glassdoor est.),Lockheed Martin is seeking a Circuit and FPGA ...,2.9,Albin Engineering Services\r\n2.9,"Fort Worth, TX","Santa Clara, CA",51 to 200 employees,1993.0,Private,Staffing & Outsourcing,Business Services,$5 to $10 million (USD),NaN,NaN
2525,Sr. Big Data Engineer,$76K-$128K (Glassdoor est.),Posting Title Principal Data Engineer (Contrac...,3.7,Cincinnati Bell Technology Solutions\r\n3.7,"Irving, TX","Cincinnati, OH",501 to 1000 employees,NaN,Private,IT Services,Information Technology,$100 to $500 million (USD),NaN,NaN
2526,Senior Big Data / ETL Engineer,$76K-$128K (Glassdoor est.),"Your Opportunity\r\n\r\nCharles Schwab & Co., ...",3.5,Schwab\r\n3.5,"Westlake, TX","Dettenheim, Germany",Unknown,NaN,Public,NaN,NaN,Unknown / Non-Applicable,NaN,NaN
